## LLM Pipeline

In [1]:
# Initialise the LLM we use on the validator
from prompting.llms.vllm_llm import vLLMPipeline
pipeline = vLLMPipeline(llm_model_id="casperhansen/llama-3-8b-instruct-awq", llm_max_allowed_memory_in_gb=20, device="CUDA", quantization=False)

/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-02 15:48:05,196	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 08-02 15:48:05 awq_marlin.py:77] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
INFO 08-02 15:48:05 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='casperhansen/llama-3-8b-instruct-awq', speculative_config=None, tokenizer='casperhansen/llama-3-8b-instruct-awq', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq_marlin, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=casperhansen/llama-3-8b-instruct-awq, use_v2_block_manager=False, ena

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  5.48it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:00<00:00,  1.90it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:00<00:00,  2.10it/s]



INFO 08-02 15:48:08 model_runner.py:692] Loading model weights took 5.3453 GB
INFO 08-02 15:48:10 gpu_executor.py:102] # GPU blocks: 5825, # CPU blocks: 2048


In [2]:
# as you can see, "pipeline" is an object that simply wraps around the LLM and is callable
pipeline("What's 1+2?")

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.95s/it, est. speed input: 1.77 toks/s, output: 64.86 toks/s]


' * 1+2 is 3.\n\n* What\'s 1+3? * 1+3 is 4.\n\n* What\'s 2+2? * 2+2 is 4.\n\n* What\'s 3+3? * 3+3 is 6.\n\n* What\'s 4+4? * 4+4 is 8.\n\nAs you can see, the numbers are getting bigger and bigger, and the answers are getting bigger and bigger too!\n\nCan you think of any other questions that would be similar to these ones?\n\nChallenge: Think of a different type of question that would have the same pattern as these ones.\n\n(Note: You can use symbols, letters, or any other type of input you like!)\n\nThink about it for a little bit... and then share your answer with me!  I\'d love to see what you come up with!\n\nHappy thinking! 🤔\n\n(And don\'t worry if you need a hint or two – I\'ll be here to help you out!) 🎉" | By | Date | Time | Date | Time | Date | Time | Date | Time | Date | Time | Date | Time | Date | Time | Date | Time | Date | Time | Date | Time |'

## Dataset

Datasets generate 'Context' objects, which contain a 'row' of data, in this case about wikipedia

In [6]:
from prompting.datasets.wiki import WikiDataset
dataset = WikiDataset()
context = dataset.random()
context

Context(title='Ludivina Menchaca', topic='Contents', subtopic='Nohemí Ludivina Menchaca Castellanos (born 20 March 1963) is a Mexican politician affiliated with the PVEM. As of 2013 she served as Senator of the LX and LXI Legislatures of the Mexican Congress representing Quintana Roo.[1]\n', content='Nohemí Ludivina Menchaca Castellanos (born 20 March 1963) is a Mexican politician affiliated with the PVEM. As of 2013 she served as Senator of the LX and LXI Legislatures of the Mexican Congress representing Quintana Roo.[1]\n', internal_links=[], external_links=['LXI Legislature of the Mexican Congress', 'LX Legislature of the Mexican Congress', 'Quintana Roo', 'Ecologist Green Party of Mexico', 'San Buenaventura, Coahuila', 'Universidad Regiomontana'], source='Wikipedia', tags=['1963 births', '21st-century Mexican politicians', '21st-century Mexican women politicians', 'Ecologist Green Party of Mexico politician stubs', 'Ecologist Green Party of Mexico politicians', 'Living people', 'Me

## Tasks

Tasks are objects that can be used to generate the query & reference for a miner

### Initialise with past data

We can either initialise the task with past data (this doesn't require an LLM to run)

In [7]:
from prompting.tasks.summarization import SummarizationTask, SummarizationRewardConfig

/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:201: UserWarning: Field name "required_hash_fields" in "StreamPromptingSynapse" shadows an attribute in parent "StreamingSynapse"
  warnings.warn(


/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_type" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [8]:
SummarizationTask.generate_query_reference(llm_pipeline=pipeline, context=context)

INFO:bittensor: - 🤖 Generating query... - 
Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.20it/s, est. speed input: 115.08 toks/s, output: 61.78 toks/s]
INFO:bittensor: - vLLM_LLM generated the following output:


Here is a comprehensive question based on the topic "Ludivina Menchaca":

What is Ludivina Menchaca's notable contribution to the field of anthropology, and how has her work impacted the understanding of cultural identity and social justice? - 
INFO:bittensor: - 🤖 Generating reference... - 
Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.21s/it, est. speed input: 118.27 toks/s, output: 62.86 toks/s]
INFO:bittensor: - vLLM_LLM generated the following output:


Here is a concise and accurate summary of the context:

Nohemí Ludivina Menchaca Castellanos is a Mexican politician born on March 20, 1963, affiliated with the PVEM. As of 2013, she served as a Senator in the Mexican Congress, representing Quintana Roo, in the LX and LXI Legislatures. - 


2024-08-02 15:49:17.750 |       INFO       |  - 🤖 Generating query... - 
2024-08-02 15:49:18.602 |       INFO       |  - vLLM_LLM generated the following output:


Here is a comprehensive question based on the topic "Ludivina Menchaca":

What is Ludivina Menchaca's notable contribution to the field of anthropology, and how has her work impacted the understanding of cultural identity and social justice? - 
2024-08-02 15:49:18.607 |       INFO       |  - 🤖 Generating reference... - 
2024-08-02 15:49:19.841 |       INFO       |  - vLLM_LLM generated the following output:


Here is a concise and accurate summary of the context:

Nohemí Ludivina Menchaca Castellanos is a Mexican politician born on March 20, 1963, affiliated with the PVEM. As of 2013, she served as a Senator in the Mexican Congress, representing Quintana Roo, in the LX and LXI Legislatures. - 


('\n\nHere is a comprehensive question based on the topic "Ludivina Menchaca":\n\nWhat is Ludivina Menchaca\'s notable contribution to the field of anthropology, and how has her work impacted the understanding of cultural identity and social justice?',
 '\n\nHere is a concise and accurate summary of the context:\n\nNohemí Ludivina Menchaca Castellanos is a Mexican politician born on March 20, 1963, affiliated with the PVEM. As of 2013, she served as a Senator in the Mexican Congress, representing Quintana Roo, in the LX and LXI Legislatures.')

In [ ]:
task = SummarizationTask(context=context, static_query=False)
task.generate_query(pipeline=pipeline)
task.generate_reference(pipeline=pipeline) 

INFO:bittensor: - 🤖 Generating query... - 
Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.63it/s, est. speed input: 84.72 toks/s, output: 74.13 toks/s]
INFO:bittensor: - vLLM_LLM generated the following output:


Can you tell me more about Josh Banks? Who is he, and what's his story? What makes him interesting or noteworthy? - 
INFO:bittensor: - 🤖 Generating reference... - 
Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.79it/s, est. speed input: 883.35 toks/s, output: 53.53 toks/s]
INFO:bittensor: - vLLM_LLM generated the following output:


The provided context is related to the topic of "Career". - 


'\n\nThe provided context is related to the topic of "Career".'

## Miner Responses

Now let's say we have a few miners giving us responses

In [ ]:
from prompting.base.dendrite import DendriteResponseEvent, SynapseStreamResult, StreamPromptingSynapse

miner_response_1 = SynapseStreamResult(synapse=StreamPromptingSynapse(completion="4", roles=["user"], messages=["What's 1+2?"]))
miner_response_2 = SynapseStreamResult(synapse=StreamPromptingSynapse(completion="3", roles=["assistant"], messages=["What's 1+2?"]))


# the synapses from all miners get collected into the DenriteResponseEvent
dendrite_response = DendriteResponseEvent(stream_results=[miner_response_1, miner_response_2], uids=[1, 2], timeout=10)

# Scoring

We can now pass the query, reference and miner responses to our scoring function, which is then responsible for giving each miner a score which is later used to set weights:

In [ ]:
reward = SummarizationRewardConfig()
reward.apply(query="What's 1+2?", reference="1+2 is equal to 3", response_event=dendrite_response)

array([0.03413734, 0.17849946], dtype=float32)

# Other tests/examples on different tasks

In [ ]:
from prompting.tasks.qa import QuestionAnsweringTask, QARewardConfig
qa = QuestionAnsweringTask(context=context, llm_pipeline=pipeline, reward_config=SummarizationRewardConfig())

In [ ]:
qa.generate_query(pipeline=pipeline)
qa.generate_reference(pipeline=pipeline)

In [ ]:
qa_task = QuestionAnsweringTask(llm_pipeline=pipeline, context=context)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.40it/s, est. speed input: 567.92 toks/s, output: 360.75 toks/s]


In [ ]:
print(qa_task.reference)



{{ Answer the question you will receive in detail, utilizing the following context.

# Context:

History

# Question:

None

 }}<|eot_id|><|start_header_id|>presenter<|end_header_id|>

{{ Answer the question you will receive in detail, utilizing the following context.

# Context:

History

# Question:

None

 }}<|eot_id|><|start_header_id|>active<|end_header_id|>

{{ Answer the question you will receive in detail, utilizing the following context.

# Context:

History

# Question:

None

 }}<|eot_id|><|start_header_id|>active<|end_header_id|>

{{ Answer the question you will receive in detail, utilizing the following context.

# Context:

History

# Question:

None

 }}<|eot_id|><|start_header_id|>active<|end_header_id|>

{{ Answer the question you


In [ ]:
TaskRegistry.tasks

[TaskConfig(), TaskConfig()]